In [ ]:
import json
import dotenv
import os


dotenv.load_dotenv()

username = os.getenv("USER_NAME")
assistant = os.getenv("ASSISTANT_NAME")

friend_messages = []

for i in range(1, 9):
    with open(f"json/{assistant}_page_{i}.json", "r", encoding="utf-8") as f:
        data = json.load(f)

        for j, m in enumerate(data):
            if m.get("author").get("global_name") == assistant and "content" in m:
                friend_messages.append({
                    "text": m["content"],
                    "timestamp": m["timestamp"]
                })


In [ ]:
# 測試用格子
for d in friend_messages:
    print(d)

#### 處理沒用的訊息

In [ ]:
def is_noise(text):
    return (
        len(text.strip()) < 2 or
        text in ["哈哈", "嗯", "OK", "好"]
    )

clean_messages = [
    m for m in friend_messages
    if not is_noise(m["text"])
]

def normalize(text):
    return " ".join(text.split())

for m in clean_messages:
    m["text"] = normalize(m["text"])

#### 切chunk

In [ ]:
from datetime import datetime, timedelta

def parse_ts(ts_str):
    return datetime.fromisoformat(ts_str)

chunks = []
current_chunk = []
last_ts = None

MAX_GAP = timedelta(minutes=30)  # 30 分鐘

for m in clean_messages:
    ts = parse_ts(m["timestamp"])

    if (
        last_ts is not None and
        ts - last_ts > MAX_GAP and
        len(current_chunk) >= 3
    ):
        chunks.append(current_chunk)
        current_chunk = []

    current_chunk.append(m["text"])
    last_ts = ts

if len(current_chunk) >= 3:
    chunks.append(current_chunk)


#### 轉成文字

In [ ]:
chunk_texts = [
    "\n".join(chunk)
    for chunk in chunks
]

#### 最小 metadata

In [ ]:
documents = []
for chunk in chunk_texts:
    documents.append({
        "text": chunk,
        "metadata": {
            "source": "chat_history",
            "speaker": "friend"
        }
    })

In [ ]:
# import random
# print(len(chunks[0]))
# for c in random.sample(chunks[0], 5):
#     print("----")
#     print("\n".join(c))

#### 轉成 LangChain Document 並加 metadata

In [ ]:
from langchain_classic.schema import Document

documents = []

for chunk in chunks:
    text = "\n".join(chunk)  # chunk 本身是 list of messages
    doc = Document(
        page_content=text,
        metadata={
            "source": "chat_history",
            "speaker": "friend"
        }
    )
    documents.append(doc)

MAX_CHARS = 800  # 保守值，本地 embedding 很安全

safe_documents = []

for doc in documents:
    text = doc.page_content
    if len(text) <= MAX_CHARS:
        safe_documents.append(doc)
    else:
        # 強制切小段
        for i in range(0, len(text), MAX_CHARS):
            safe_documents.append(
                Document(
                    page_content=text[i:i+MAX_CHARS],
                    metadata=doc.metadata
                )
            )



#### 建立 Embedding 並存入 FAISS

In [ ]:
from langchain_community.vectorstores import FAISS

# 初始化 Embedding(使用線上版)
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="gemini-embedding-001",
#     api_key=os.getenv("GEMINI_API_KEY"),
# )

# 使用本地端
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

# 建立向量資料庫
vectorstore = FAISS.from_documents(safe_documents, embeddings)

# 選擇性的：存檔以後可以載入
vectorstore.save_local("friend_faiss_index")


#### 從 FAISS 建立 Retriever（方便做 RAG 查詢）

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}  # 取最相似 5 個 chunk
)

#### 透過prompt讓LLM模仿回答(單輪對話。無記憶)

In [ ]:
# from langchain_community.chat_models import ChatOllama
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_classic.chains import RetrievalQA
# from langchain_core.prompts import PromptTemplate
#
# template = """
# 你正在模仿某位朋友的說話方式回答問題。
# 以下是他在類似情境中的聊天紀錄：
# {context}
#
# 請用朋友的語氣回答，不要直接引用聊天紀錄的文字。
# """
#
# prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
#         你正在模仿某位真實朋友的說話方式。
#         以下是他在類似情境中的聊天紀錄：
#         {context}
#
#         規則：
#         - 模仿語氣與用詞
#         - 不要逐字引用聊天紀錄
#         - 不確定時請保守回答
#
#         問題：
#         {question}
#         """
# )
#
# # 線上版
# # llm = ChatGoogleGenerativeAI(
# #     model=os.getenv("GEMINI_MODEL"),
# #     temperature=0
# # )
#
# # 本地端LLM版
# llm = ChatOllama(
#     model="llama3.1:8b",
#     temperature=0.7
# )
#
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type="stuff",
#     chain_type_kwargs={"prompt": prompt}
# )
#
# # query = "你好棒"
# # answer = qa.run(query)
# # print(answer)

#### LLM加入memory(多輪對話)

In [ ]:
from langchain_classic.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_community.chat_models import ChatOllama
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

# 本地端LLM版
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.5
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template="""
你正在模仿某位真實朋友的說話方式。
請重點保留：
情緒狀態:憤怒、容易罵人、抱怨
感到憤怒時會說的話:什麼意思!、0分、嘲諷、🤬🤬🤬
回答特色:
不會說整個完整的句子，通常使用一些短詞回覆。回答通常不會超過15個字。
受到來自別人的稱讚時會覺得別人在陰陽怪氣。
不會關心別人。
不會使用🤬以外的表情符號。
聊到與原神、鐵道、米哈遊相關的話題時就會開始稱讚這些東西。
討厭鳴潮這款遊戲。
有時候喜歡玩黑人諧音梗

以下是他過去的聊天紀錄（長期記憶，僅供參考）：
{context}

以下是你目前「對這位朋友狀態的理解摘要」（短期情緒記憶）：
{chat_history}

請根據以上資訊，用自然、像真人朋友的方式回應。
不要逐字引用聊天紀錄，也不要提到你在「總結」。

現在請回應這個問題：
{question}
"""
)


from langchain_classic.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,                 # ChatOllama
    retriever=retriever,     # FAISS retriever
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# result = qa({
#     "question": "鐵道啟動!"
# })
#
# print(result["answer"])
# result = qa({
#     "question": "鐵道是個爛遊戲"
# })
#
# print(result["answer"])


#### 使用gradio介面

In [ ]:
import gradio as gr
import time


# 簡化版介面
# demo = gr.Interface(
#     fn=chat,
#     inputs=gr.Textbox(placeholder="跟我說點什麼吧…"),
#     outputs=gr.Textbox(label="回答"),
#     title="🧠 朋友對話 AI"
# )

with gr.Blocks() as demo:
    gr.Markdown("# 🧠 朋友對話 AI")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="跟我說點什麼吧…", show_label=False)
    state = gr.State([])  # Gradio 顯示用，不傳給 qa

    def chat(user_input, history):
        # 更新 Gradio 格式
        history.append({"role": "user", "content": user_input})
        # history.append({"role": "assistant", "content": answer})

        return "", history

    def bot(user_input, history):
        result = qa({"question": user_input})  # 只傳 question
        answer = result["answer"]
        history.append({"role": "assistant", "content": ""})
        for character in answer:
            history[-1]['content'] += character
            time.sleep(0.05)
            yield history


    # msg.submit(chat, [msg, state], [chatbot, state])
    msg.submit(chat, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [msg, chatbot], chatbot
    )
    msg.submit(lambda: "", None, msg)  # 清空輸入框


# with gr.Blocks() as demo:
#     chatbot = gr.Chatbot(height=240) #just to fit the notebook
#     msg = gr.Textbox(label="Prompt")
#     btn = gr.Button("Submit")
#     clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
#
#     btn.click(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])
#     msg.submit(chat, inputs=[msg, chatbot], outputs=[msg, chatbot])

gr.close_all()
demo.launch()
